In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import re

In [2]:
from datetime import datetime

In [3]:
def remove_all(string):
    pattern = re.compile(r'[А-Яа-яёЁ0-9 ]+')
    return pattern.findall(string)[0].strip()

In [4]:
def find_all_theaters_KARO(theatres):
    dicti = {}
    metro_class = 'cinemalist__cinema-item__metro__station-list__station-item'
    for theater in theatres:
        dicti[theater.findAll('h4')[0].text[1:]] = {
            'metro': [remove_all(i.text) for i in theater.findAll('li', class_=metro_class)], 
            'address': theater.findAll('p')[0].text.split('+')[0].strip(),
            'phone': '+' + theater.findAll('p')[0].text.split('+')[-1],
            'data-id': theater['data-id']
        }
    return dicti

In [8]:
url = "https://karofilm.ru"
url_theaters = url + "/theatres"

In [9]:
r = requests.get(url_theaters)
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "html.parser")
    theatres = soup.findAll('li', class_='cinemalist__cinema-item')
    karo_theatres = find_all_theaters_KARO(theatres)
else:
    print("Страница не найдена")

In [10]:
conn = sqlite3.connect("cinemabot.db")
cursor = conn.cursor()

In [7]:
cursor.execute("""CREATE TABLE IF NOT EXISTS brand(
                id integer PRIMARY KEY,
                name text NOT NULL
                )""")

In [5]:
cursor.execute("""CREATE TABLE IF NOT EXISTS cinema_halls(
                id integer PRIMARY KEY,
                brand_id integer Not NULL,
                name text NOT NULL,
                address text NOT NULL,
                metro text NOT NULL,
                phone text NOT NULL,
                FOREIGN KEY (brand_id) REFERENCES brand(id)
                )""")

In [40]:
cursor.execute("""CREATE TABLE IF NOT EXISTS cinemas(
                id integer PRIMARY KEY,
                name text NOT NULL,
                duration integer NOT NULL,
                genres text NOT NULL
                )""")

In [41]:
cursor.execute("""CREATE TABLE IF NOT EXISTS sessions(
                id integer PRIMARY KEY,
                cinema_id integer Not NULL,
                hall_id integer Not NULL,
                name text NOT NULL,
                date date NOT NULL,
                time time NOT NULL,
                price integer NOT NULL,
                FOREIGN KEY (cinema_id) REFERENCES cinemas(id),
                FOREIGN KEY (hall_id) REFERENCES cinema_halls(id),
                FOREIGN KEY (name) REFERENCES cinemas(id)
                )""")

In [12]:
cursor.execute("insert into brand values (1, 'КАРО')")
cursor.execute("insert into brand values (2, 'Синема Парк')")
cursor.execute("insert into brand values (3, 'Люксор')")
conn.commit()

In [13]:
cursor.execute('select * from brand').fetchall()

[(1, 'КАРО'), (2, 'Синема Парк'), (3, 'Люксор')]

In [10]:
for key, item in karo_theatres.items():
    elements=[1,key,item['address'],', '.join(item['metro']),item['phone']]
    cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({},'{}','{}','{}','{}')".format(*elements))
conn.commit()

In [42]:
r2 = requests.get(url)
if r2.status_code == 200:
    soup = BeautifulSoup(r2.text, 'html.parser')
    films = soup.findAll('div', class_ = 'afisha-item-inner')
    for film in films:
        film_name = film.findAll('h3', class_ = "afisha-title")[0].text
        film_duration = film.findAll('span', class_ = 'afisha-duration-time')[0].text
        try:
            film_genres = film.findAll('p', class_ = 'afisha-genre')[0].text
        except IndexError:
            film_genres = 'undefined'
        elements = [film_name, film_duration, film_genres]
        try:
            cursor.execute("insert into cinemas (name, duration, genres) values ('{}', '{}', '{}')".format(*elements))
        except sqlite3.OperationalError:
            continue
    conn.commit()
else:
    print('page not found')

In [5]:
def cinema_id_get(name, cinemas):
    for el in cinemas:
        if name == el[1]:
            return el[0]
    for el in cinemas:
        if (name in el[1]) or (el[1] in name):
            return el[0]

In [44]:
films_class = 'cinema-page-item__schedule__row'
table_class = 'cinema-page-item__schedule__row__board-row'
left_class = table_class + '__left'
rignt_class = table_class + '__right'
date_class = 'widget-select'
for theater in karo_theatres:
    dates = {}
    url_theater_id = url_theaters + '?id=' + karo_theatres[theater]['data-id']
    r3 = requests.get(url_theater_id)
    if r3.status_code == 200:
        date_parser = BeautifulSoup(r3.text,'html.parser')
        date_list = date_parser.findAll('select',class_ = date_class)[0]
        date_list = [i['data-id'] for i in date_list.findAll('option')]
        for date in date_list: 
            url_theater_id_date = url_theater_id + '&date=' + date
            r4 = requests.get(url_theater_id_date)
            session = {}
            if r4.status_code == 200:
                films_parser = BeautifulSoup(r4.text,'html.parser')
                films_list = films_parser.findAll('div', class_ = films_class)
                for film in films_list:
                    name = film.findAll('h3')
                    if name:
                        name = name[0].text.split(', ')
                        session_time = {}
                        session_time['age'] = name[1]
                        for i in film.findAll('div', class_ = table_class):
                            time_D = i.findAll('div', class_ = left_class)[0].text.strip()
                            time = i.findAll('div', class_ = rignt_class)[0].findAll('a')
                            time = [j.text for j in time]
                            session_time[time_D] = time
                            for time_element in time:
                                cinema_id = cinema_id_get(name[0], cursor.execute('select * from cinemas').fetchall())
                                if cinema_id == None:
                                    cinema_id = 0
                                hall_id = cursor.execute(f'select id from cinema_halls where name=\'{theater}\'').fetchall()[0][0]
                                values = [cinema_id, hall_id, name[0], date, time_element, 0]
                                try:
                                    cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
                                except sqlite3.OperationalError:
                                    continue
                        session[name[0]] = session_time
            else:
                print('Ненаход даты, url=', url_theater_id_date)
            dates[date]=session
    else:
        print('Ненаход кинотеатра, url=', url_theater_id)
    karo_theatres[theater]['dates'] = dates
conn.commit()

In [6]:
url = "https://kinoteatr.ru"
url_theatres = url + "/raspisanie-kinoteatrov"
url_films = url + "/kinoafisha"

In [7]:
def tab_delete(name):
    pattern = re.compile(r'[А-Яа-яёЁ0-9.,-:]+')
    res = ' '.join(pattern.findall(name))
    return res

def number(price):
    pattern = re.compile(r'[0-9]+')
    res = ' '.join(pattern.findall(price))
    return res

def tab_delete_name(name):
    pattern = re.compile(r'\w+[:,.!?]?')
    res = ' '.join(pattern.findall(name))
    return res

def date_maker(date):
    pattern = re.compile(r'[0-9]+-[0-9]+-[0-9]+')
    res = ', '.join(pattern.findall(date))
    return res

In [8]:
r = requests.get(url_theatres)
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "html.parser")
    theatres = soup.findAll('div', class_ = "col-md-12 cinema_card")
    for theatre in theatres:
        name = tab_delete(theatre.findAll('h3', class_ = "title movie_card_title")[0].text)
        metro = [tab_delete(i) for i in theatre.findAll('span', class_ = "sub_title")[1] if ('\n' in i) and (len(i) > 2)]
        address = tab_delete(theatre.findAll('span', class_ = "sub_title")[0].text)
        phone = 'unknown'
        values = [2, name, address, ', '.join(metro), phone]
        #cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({},'{}','{}','{}','{}')".format(*values))
    #conn.commit()
else:
    print('page not found')

In [11]:
r2 = requests.get(url_films)
if r2.status_code == 200:
    soup = BeautifulSoup(r2.text, "html.parser")
    films = soup.findAll('div', class_ = "col-md-2 col-sm-6 col-xs-12 movie_card")
    for film in films:
        count = 0
        name = tab_delete_name(film.findAll('span', class_ = "movie_card_header title")[0].text)
        name_2 = cursor.execute('select name from cinemas').fetchall()
        for i in name_2:
            if name in i[0]:
                count += 1
        if count == 0:
            try:
                duration = tab_delete(film.findAll('span', class_ = "hide-md title")[0].text)
            except IndexError:
                duration = '0 мин'
            #print(duration)
            genres = [tab_delete(i.text) for i in film.findAll('span', class_ = "hidden", itemprop = "genre")]
            values = [name, duration.split()[0], ', '.join(genres)]
            #try:
                #cursor.execute("insert into cinemas (name, duration, genres) values ('{}', {},'{}')".format(*values))
            #except sqlite3.OperationalError:
                #continue
    conn.commit()
else:
    print('page not found')

In [12]:
halls = ['/belaya-dacha', '/butovo-mall', '/waypark', '/global-city', '/gorizont', '/evropa', '/zelenopark', '/kaluzhskij', '/kutuzovskiy', '/ladoga', '/lefortovo', '/metropolis', '/michurinsky', '/mozhayka', '/oblaka', '/polezhaevskiy', '/5avenu', '/rivera', '/semenovsky', '/city', '/tepliy-stan', '/filion', '/mega-himki', '/cdm', '/chertanovo']

In [17]:
k = 0
hall_id = 17
cinema_id = 0
for theatre in theatres:
    url_table = url_theatres + halls[k]
    k += 1
    r3 = requests.get(url_table)
    if r3.status_code == 200:
        dates = []
        soup = BeautifulSoup(r3.text, "html.parser")
        date_parser = soup.findAll('input', class_ = "single_card_date_input")
        for date in date_maker(str(date_parser)).split(', '):
            if date not in dates:
                dates.append(date)
        for date in dates:
            url_date = url_table + '/?date=' + date
            r4 = requests.get(url_date)
            if r4.status_code == 200:
                schedule = BeautifulSoup(r4.text, "html.parser")
                films = schedule.findAll('div', class_ = 'shedule_movie')
                for film in films:
                    money = []
                    n = 0
                    title = film.findAll('span', class_ = "movie_card_header title")
                    name = tab_delete_name(title[0].text)
                    cinema_id = cinema_id_get(name, cursor.execute('select * from cinemas').fetchall())
                    sessions = film.findAll('span', class_ = "shedule_session_time")
                    prices = film.findAll('span', class_ = "shedule_session_price")
                    for price in prices:
                        money.append(number(price.text))
                    for session in sessions:
                        time = tab_delete(session.text)
                        try:
                            values = [cinema_id, hall_id, name, date, time, money[n]]
                        except IndexError:
                            values = [cinema_id, hall_id, name, date, time, 0]
                        n += 1
                        print(values, n, len(money))
                        #try:
                         #   cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
                        #except sqlite3.OperationalError:
                           # continue
            else:
                print('sessions not found')
    else:
        print('page not found')
    hall_id += 1
    conn.commit()

[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '19:10', '550'] 1 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '19:35', '710'] 2 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '20:00', '990'] 3 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '20:30', '550'] 4 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '21:00', '860'] 5 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '21:30', '550'] 6 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '22:00', '640'] 7 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '22:25', '550'] 8 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '22:50', '710'] 9 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '23:15', '990'] 10 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '23:45', '550'] 11 13
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '00:15', '860'] 12 13
[3, 17, 'Звёздные Войны: Скайуокер. В

[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '00:15', '860'] 1 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '00:45', '550'] 2 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '10:15', '580'] 3 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '10:45', '300'] 4 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '11:15', '590'] 5 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '11:45', '300'] 6 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '12:15', '350'] 7 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '13:05', '560'] 8 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '13:30', '730'] 9 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '14:00', '370'] 10 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '14:30', '710'] 11 29
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-23', '15:30', '440'] 12 29
[3, 17, 'Звёздные Войны: Скайуокер. В

[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '10:15', '580'] 1 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '10:45', '300'] 2 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '11:15', '590'] 3 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '11:45', '300'] 4 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '12:15', '350'] 5 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '13:05', '560'] 6 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '13:30', '730'] 7 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '14:00', '370'] 8 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '14:30', '710'] 9 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '15:30', '440'] 10 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '16:20', '560'] 11 27
[3, 17, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '16:45', '730'] 12 27
[3, 17, 'Звёздные Войны: Скайуокер. В

[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '19:15', '450'] 14 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '20:00', '410'] 15 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '20:50', '480'] 16 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '21:15', '450'] 17 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '21:40', '410'] 18 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '22:30', '450'] 19 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '23:15', '410'] 20 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '00:05', '480'] 21 22
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-22', '00:55', '410'] 22 22
[9, 18, 'Фиксики против кработов', '2019-12-22', '10:50', '260'] 1 8
[9, 18, 'Фиксики против кработов', '2019-12-22', '11:35', '260'] 2 8
[9, 18, 'Фиксики против кработов', '2019-12-22', '12:40', '340'] 3 8
[9, 18, 'Фиксики против кработов', '2019-12-22', '14:30', '430']

[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '16:45', '260'] 9 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '17:35', '330'] 10 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '18:25', '310'] 11 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '19:15', '350'] 12 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '20:00', '310'] 13 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '20:50', '380'] 14 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '21:40', '310'] 15 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '22:30', '350'] 16 17
[3, 18, 'Звёздные Войны: Скайуокер. Восход', '2019-12-25', '23:15', '310'] 17 17
[8, 18, 'Полицейский с Рублевки. Новогодний беспредел 2', '2019-12-25', '10:15', '170'] 1 9
[8, 18, 'Полицейский с Рублевки. Новогодний беспредел 2', '2019-12-25', '12:15', '170'] 2 9
[8, 18, 'Полицейский с Рублевки. Новогодний беспредел 2', '2019-12-25', '14:15', '200'] 

...

[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '19:15', '670'] 1 7
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '20:00', '660'] 2 7
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '20:45', '610'] 3 7
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '21:40', '610'] 4 7
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '22:30', '670'] 5 7
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '23:15', '660'] 6 7
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-21', '23:55', '610'] 7 7
[37, 40, 'Достать ножи', '2019-12-21', '19:15', '550'] 1 3
[37, 40, 'Достать ножи', '2019-12-21', '22:20', '550'] 2 3
[37, 40, 'Достать ножи', '2019-12-21', '00:55', '550'] 3 3
[22, 40, 'Сиротский Бруклин', '2019-12-21', '20:00', '550'] 1 2
[22, 40, 'Сиротский Бруклин', '2019-12-21', '22:05', '800'] 2 2
[8, 40, 'Полицейский с Рублевки. Новогодний беспредел 2', '2019-12-21', '21:55', '550'] 1 1
[10, 40, 'Джуманджи: Новый уровень', '2019-12-21',

[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '00:10', '570'] 1 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '10:15', '450'] 2 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '11:10', '460'] 3 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '12:00', '400'] 4 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '12:45', '610'] 5 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '13:30', '460'] 6 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '14:25', '470'] 7 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '16:00', '610'] 8 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '16:45', '460'] 9 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '17:40', '470'] 10 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '18:25', '510'] 11 17
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-24', '20:00', '560'] 12 17
[3, 40, 'Звёздные Войны: Скайуокер. В

[None, 40, 'Союз спасения', '2019-12-27', '00:30', '590'] 1 11
[None, 40, 'Союз спасения', '2019-12-27', '10:15', '430'] 2 11
[None, 40, 'Союз спасения', '2019-12-27', '12:00', '420'] 3 11
[None, 40, 'Союз спасения', '2019-12-27', '13:00', '480'] 4 11
[None, 40, 'Союз спасения', '2019-12-27', '14:45', '470'] 5 11
[None, 40, 'Союз спасения', '2019-12-27', '17:30', '470'] 6 11
[None, 40, 'Союз спасения', '2019-12-27', '19:00', '590'] 7 11
[None, 40, 'Союз спасения', '2019-12-27', '20:15', '580'] 8 11
[None, 40, 'Союз спасения', '2019-12-27', '21:45', '590'] 9 11
[None, 40, 'Союз спасения', '2019-12-27', '23:00', '580'] 10 11
[None, 40, 'Союз спасения', '2019-12-27', '00:30', '590'] 11 11
[3, 40, 'Звёздные Войны: Скайуокер. Восход', '2019-12-27', '15:50', '530'] 1 1
[137, 40, 'TheatreHD: Комеди Франсез: Блоха в ухе', '2019-12-27', '19:30', '600'] 1 1
[None, 40, 'Союз спасения', '2019-12-28', '00:30', '590'] 1 6
[None, 40, 'Союз спасения', '2019-12-28', '12:00', '420'] 2 6
[None, 40, 'Союз

[8, 41, 'Полицейский с Рублевки. Новогодний беспредел 2', '2019-12-22', '17:20', '310'] 1 3
[8, 41, 'Полицейский с Рублевки. Новогодний беспредел 2', '2019-12-22', '21:40', '340'] 2 3
[8, 41, 'Полицейский с Рублевки. Новогодний беспредел 2', '2019-12-22', '00:25', '340'] 3 3
[10, 41, 'Джуманджи: Новый уровень', '2019-12-22', '10:20', '210'] 1 3
[10, 41, 'Джуманджи: Новый уровень', '2019-12-22', '19:05', '340'] 2 3
[10, 41, 'Джуманджи: Новый уровень', '2019-12-22', '23:40', '340'] 3 3
[37, 41, 'Достать ножи', '2019-12-22', '18:45', '340'] 1 2
[37, 41, 'Достать ножи', '2019-12-22', '00:20', '340'] 2 2
[None, 41, 'Черное Рождество', '2019-12-22', '22:20', '340'] 1 1
[25, 41, 'Рождество на двоих', '2019-12-22', '15:15', '310'] 1 1
[30, 41, 'Щенячий патруль: Мегащенки и отважные птенцы', '2019-12-22', '10:30', '120'] 1 1
[40, 41, 'Ford против Ferrari', '2019-12-22', '21:25', '340'] 1 1
[20, 41, 'Ржев', '2019-12-22', '12:55', '310'] 1 1
[170, 41, 'МУЛЬТ в кино. Выпуск 109', '2019-12-22', '10

[145, 41, 'TheatreHD: Жизель', '2020-01-26', '18:00', '650'] 1 1
[None, 41, 'TheatreHD: Порги и Бесс', '2020-02-01', '20:55', '1000'] 1 1
[146, 41, 'TheatreHD: Лебединое озеро', '2020-02-23', '18:00', '650'] 1 1
[None, 41, 'TheatreHD: Агриппина', '2020-02-29', '20:55', '1000'] 1 1
[None, 41, 'TheatreHD: Летучий голландец', '2020-03-14', '19:55', '1000'] 1 1
[None, 41, 'TheatreHD: БТ: Ромео и Джульетта', '2020-03-29', '18:00', '650'] 1 1
[None, 41, 'TheatreHD: Тоска', '2020-04-11', '19:55', '1000'] 1 1
[147, 41, 'TheatreHD: Драгоценности', '2020-04-19', '18:00', '650'] 1 1
[None, 41, 'TheatreHD: Мария Стюарт', '2020-05-09', '19:55', '1000'] 1 1


In [14]:
values1 = [3, 'Люксор Весна', '84-й км МКАД, ТЦ "Весна"', 'Алтуфьево', 'unknown']
values2 = [3, 'Люксор Гудзон', 'Каширское шоссе, д. 14', 'Варшавская, Каховская', 'unknown']
cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({}, '{}', '{}', '{}', '{}')".format(*values1))
cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({}, '{}', '{}', '{}', '{}')".format(*values2))
conn.commit()

In [62]:
luxor_url = 'https://www.luxorfilm.ru/'

In [63]:
def link_finder(string):
    pattern = re.compile(r'/moscow/film/[0-9]+/[A-Za-z-0-9]+')
    res = ''.join(pattern.findall(string))
    return res

def times(time):
    pattern = re.compile(r'[0-9]?[0-9]+')
    res = pattern.findall(time)
    return res

In [64]:
r5 = requests.get(luxor_url)
if r5.status_code == 200:
    filmlist = BeautifulSoup(r5.text, 'html.parser')
    films = filmlist.findAll('li', class_ = "film")
    for film in films:
        count = 0
        title = film.findAll('span')
        if len(title) == 1:
            name = title[0].text
        elif len(title) >= 2:
            name = title[0].text + ': ' + title[1].text
        else:
            name = None
        if name:
            name_2 = cursor.execute('select name from cinemas').fetchall()
            for i in name_2:
                if name in i[0] or i[0] in name:
                    count += 1
            if count == 0:    
                link = luxor_url + link_finder(str(film.findAll('a', class_ = 'film-page-link')[0]))
                r6 = requests.get(link)
                if r6.status_code == 200:
                    page = BeautifulSoup(r6.text, 'html.parser')
                    details = page.findAll('td')
                    duration = times(details[7].text)
                    time = int(duration[0])*60 + int(duration[1])
                    genres = details[5].text
                    values = [name, time, genres]
                    try:
                        cursor.execute("insert into cinemas (name, duration, genres) values ('{}', {},'{}')".format(*values))
                    except sqlite3.OperationalError:
                        continue
                else:
                    print('film not found')
    conn.commit()
else:
    print('page not found')

In [65]:
url1 = 'https://www.luxorfilm.ru/cinema/vesna/'
url2 = 'https://www.luxorfilm.ru/cinema/goodzone/'

In [66]:
r7 = requests.get(url1)
if r7.status_code == 200:
    page = BeautifulSoup(r7.text, 'html.parser')
    films = page.findAll('h3')
    filmlist = []
    for film in films:
        if 'href' in str(film):
            continue
        else:
            filmlist.append(film.text)
    films = page.findAll('div', class_ = "sessions_cinema_block")
    for film in films:
        cinema_id = 0
        name = filmlist.pop(0)
        cinema_id = cinema_id_get(name, cursor.execute('select * from cinemas').fetchall())
        date = datetime.today().strftime('%Y-%m-%d')
        sessions = film.findAll('a', class_ = "shedule-data")
        for session in sessions:
            info = tab_delete(session.text).split(' ')
            time = info[0]
            if len(info[6]) > 1:
                price = info[6]
            else:
                price = info[5]
            values = [cinema_id, 42, name, date, time, price]
            try:
                cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
            except sqlite3.OperationalError:
                continue
            conn.commit()

In [67]:
r8 = requests.get(url2)
if r8.status_code == 200:
    page = BeautifulSoup(r8.text, 'html.parser')
    films = page.findAll('h3')
    filmlist = []
    for film in films:
        if 'href' in str(film):
            continue
        else:
            filmlist.append(film.text)
    films = page.findAll('div', class_ = "sessions_cinema_block")
    for film in films:
        cinema_id = 0
        name = filmlist.pop(0)
        cinema_id = cinema_id_get(name, cursor.execute('select * from cinemas').fetchall())
        date = datetime.today().strftime('%Y-%m-%d')
        sessions = film.findAll('a', class_ = "shedule-data")
        for session in sessions:
            info = tab_delete(session.text).split(' ')
            time = info[0]
            if len(info[6]) > 1:
                price = info[6]
            else:
                price = info[5]
            values = [cinema_id, 43, name, date, time, price]
            try:
                cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
            except sqlite3.OperationalError:
                continue
            conn.commit()

In [68]:
#cursor.execute('drop table sessions')

In [4]:
#cursor.execute('drop table cinema_halls')

In [3]:
date = datetime.today().strftime('%Y-%m-%d')
date

'2019-12-21'